Let's first load a couple of the libraries that well be needed for the exploratory analysis.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA

The original data was downloaded and saved in ionosphere.txt and afterwards reshaped into a  .csv-file with the preparing_data.py script. The only difference between the original dataset and ionosphere_processed.csv is the format. I will use the latter one to explore some parts of the data.

In [ ]:
data = pd.read_csv('ionosphere_processed.csv', index_col=0)
data_mat = data.as_matrix()
n_rows, n_cols = data_mat.shape

The .describe() method of pandas.DataFrames computes a number of descriptive statistics for each variable in the columns of the data and gives first valuable insights.

In [9]:
data_descript = data.describe()
data_descript.iloc[:, :10]             # only looking at the first ten features for now

,0,1,2,3,4,5,6,7,8,9
count,351.000000,351,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000
mean,0.891738,0,0.641342,0.044372,0.601068,0.115889,0.550095,0.119360,0.511848,0.181345
std,0.311155,0,0.497708,0.441435,0.519862,0.460810,0.492654,0.520750,0.507066,0.483851
min,0.000000,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,1.000000,0,0.472135,-0.064735,0.412660,-0.024795,0.211310,-0.054840,0.087110,-0.048075
50%,1.000000,0,0.871110,0.016310,0.809200,0.022800,0.728730,0.014710,0.684210,0.018290
75%,1.000000,0,1.000000,0.194185,1.000000,0.334655,0.969240,0.445675,0.953240,0.534195
max,1.000000,0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


The second feature appears to be all zeros for each observation. Since it does not contain any information to predict the target variable, it can safely be dropped.

In [13]:
to_remove_cols = data_descript.columns[((data_descript.loc['mean', :] == 0) &
                                        (data_descript.loc['std', :] == 0))]
data_filt = data.loc[
  :, [col for col in data.columns if col not in to_remove_cols]
]